### Importar Librerias

In [1]:
from binance import Client

### API KEY y Cliente

In [19]:
apikey = 'o4xp0nX8Nr3RsQIAQDBs7ZZivwpoLHPZsDQU48dmWX8heBKpSgPOS0M9NZwHHbEP'
secret = 'OwjJSwLt0szX7qq62Xqd7evsc345eCLCTrYIVTdIND4HORYJ4DDz7lmGnDGCMiwq'
client = Client(apikey, secret)

### Traer Última Orden Completada

In [20]:
res = client.futures_get_all_orders()
status = "FILLED"

# Buscar el diccionario con el símbolo deseado
target_status = None
for position in res[::-1]:
    if position['status'] == status:
        target_status = position
        break

### TP Limit Según lado

In [21]:
aux = None
decimas = len(target_status["price"].split(".")[1])
if target_status["side"] == "SELL":
    aux = "BUY"
    precio_salida = str(round(float(target_status["avgPrice"])*0.993,decimas))
else:
    aux = "SELL"
    precio_salida = str(round(float(target_status["avgPrice"])*1.007,decimas))

### Crear Orden TP

In [22]:
orden_salida = client.futures_create_order(symbol = target_status["symbol"],
                            side = aux,
                            price = precio_salida,
                            quantity = target_status["executedQty"],
                            type = 'LIMIT',
                            timeinforce = 'GTC',
                            reduceOnly = True)

### Imprimir Ganancia

In [17]:
total_salida = float(orden_salida["price"])*float(orden_salida["origQty"])
total_entrada = float(target_status["cumQuote"])
ganancia = total_salida - total_entrada
comision = total_salida*0.0002 + total_entrada*0.0002 
ganancia_neta = abs(ganancia-comision)
print("La ganancia es aproximandamente:", round(ganancia_neta,2))

La ganancia es aproximandamente: 20.65


### Cancelar Ordenes Abiertas

In [18]:
client.futures_cancel_all_open_orders(symbol = target_status["symbol"])

{'code': 200, 'msg': 'The operation of cancel all open order is done.'}